In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import numpy as np

In [ ]:
datagen = ImageDataGenerator(rotation_range=20,
                            zoom_range=0.15,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.15,
                            horizontal_flip=True,
                            fill_mode="nearest")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = r'--'

CATEGORIES = ["m", "w","z"] # Intialisierung der Label
IMG_SIZE = 100 # Bildgröße

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        print(img_array)
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!

In [ ]:
### Trainingsbilder werden mit dem zugehörigen Label in einer Liste gespeichert
CATEGORIES = ["m", "w","z"] # Intialisierung der Label
DATADIR = '...' # Ordner in dem die Druckknopfbilder gespeichert sind 
training_data = []
def create_training_data():
    for category in CATEGORIES:  # Klassen: m, w, z
        path = os.path.join(DATADIR,category)  # Erstellung eines path zum Ordner
        class_num = CATEGORIES.index(category)  # Klassifikation: 0=m, 1=w, 2=z
        for img in tqdm(os.listdir(path)):  # Iteration über alle Bilder im Ordner
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # Konvertierung in array
                
                training_data.append([img_array, class_num])  # Hinzufügen zu Trainingsdaten
            except Exception as e:  
                pass    
create_training_data()
# training_data[0][i]: Druckknopfbild
# training_data[1][i]: zugehöriges Label (m, w, z)


In [ ]:
# Aufteilen des Datensatzes in Test- und Trainingsdatensatz
import random
random.shuffle(training_data) # Durchmischen des Datensatzes
X = [] # Bilder
y = [] # Label
for features,label in training_data:
    X.append(features)
    y.append(label)    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y = to_categorical(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% Trainingsdaten, 20% Testdaten

In [ ]:
# save as pickle data

import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
# open pickle data
import pickle
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

#X_train = X_train / 255
#X_test = X_test /255
X_train.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

model = Sequential() # Initialsierung des Modells
# Feature Learning aus Faltungsschichten und Max-Pooling Schichten
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(100, 100, 1), activation="relu", padding="same"))
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))
#
model.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(Conv2D(256, kernel_size=(3, 3), activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Klassfikation
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(3, activation="softmax"))
# Initalisieren der Verlustfunktion und des Optimierers
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
# Training des Modells
history = model.fit(
    X_train.reshape(-1, 100, 100, 1),
    y_train,
    epochs=10,
    batch_size=10,
validation_data=(X_test, y_test))

In [ ]:
### Validierung des neuronalen Netzes in einer Konfusionsmatrix


from sklearn.metrics import confusion_matrix
from sklearn import metrics
# Speichern der Ergebnisse den Testdatensatzes in einer Liste
y_predict = []
for i in range(len(X_test)):
    y_predict.append(model.predict(X_test[i].reshape(-1,100,100,1))) # Bestimmung des Labels mit dem zuvor trainierten gefalteten neuronalen Netz
y_predict = np.array(y_predict).reshape(-1,3)
y_predict = y_predict.round() # Liste mit vorhersageten Labels [1, 1, 2, 0, 2, ...] (0=m; 1=w; 2=z)
# Formatänderung für die scikit-learn Matrixfunktion
y_predict_c = [np.where(r==1)[0][0] for r in y_predict] # vorhergesagte Labe
y_test_c = [np.where(r==1)[0][0] for r in y_test] # reale Label
# Verwendung der scikit-learn Matrixfunktion
matrix = confusion_matrix(y_test_c,y_predict_c) # Konfusionsmatrix



### Bestimmung der Konfusionsmatrix der Testergebnisse
from sklearn.metrics import confusion_matrix
from sklearn import metrics

y_predict = y_predict.round()###



cm=confusion_matrix(y_test_c,y_predict_c)
print(cm)

In [ ]:
### VISUALSIERUNG DER SCHICHTEN

model2 = Sequential()

model2.add(Conv2D(32, 
                  kernel_size=(3, 3), 
                  activation="softmax", 
                  input_shape=(100, 100, 1), 
                  weights=model.layers[0].get_weights()))          

result = model2.predict(X_test[16].reshape(1, 100,100, 1))

In [ ]:
result[0][:, :, 0].shape

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.imshow(result[0][:, :, 0], cmap="gray_r")
plt.show()

plt.imshow(X_test[0].reshape(100,100), cmap="gray_r")
plt.show()